ICR - Identify Age_Related Conditions
=====================================

[Kaggle Competition](https://www.kaggle.com/competitions/icr-identify-age-related-conditions)

Sample submission
Based on: [gusthema/identifying-age-related-conditions-w-tfdf](https://www.kaggle.com/code/gusthema/identifying-age-related-conditions-w-tfdf)

In [16]:
!pip install sklearn 
!pip install tensorflow
!pip install tensorflow_decision_forests

# Data Exploration

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# read in data


df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test_df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
sample_submission = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')

# df = pd.read_csv('../data/train.csv')
df.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


# Data Cleaning

# Model

## Data splitting

In [19]:
from sklearn.model_selection import KFold
import tensorflow as tf
import tensorflow_decision_forests as tfdf

In [20]:
# Creates a GroupKFold with 5 splits
kf = KFold(n_splits=5, shuffle=True)

# Create list of ids for the creation of oof dataframe.
ID_LIST = df.index

# Create a dataframe of required size with zero values.
oof = pd.DataFrame(data=np.zeros((len(ID_LIST),1)), index=ID_LIST)

# Create an empty dictionary to store the models trained for each fold.
models = {}

# Create empty dict to save metrics for the models trained for each fold.
accuracy = {}
cross_entropy = {}

# Save the name of the label column to a variable.
label = "Class"

# Calculate the number of samples for each label.
neg, pos = np.bincount(df['Class'])

# Calculate total samples.
total = neg + pos

# Calculate the weight for each label.
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

FEATURE_COLUMNS = [i for i in df.columns if i not in ["Id"]]

## Model training

In [21]:
# Loop through each fold
for i, (train_index, valid_index) in enumerate(kf.split(X=df)):
        print('##### Fold',i+1)

        # Fetch values corresponding to the index 
        train_df = df.iloc[train_index]
        valid_df = df.iloc[valid_index]
        valid_ids = valid_df.index.values
        
        # Select only feature columns for training.
        train_df = train_df[FEATURE_COLUMNS]
        valid_df = valid_df[FEATURE_COLUMNS]
        
        # There's one more step required before we can train the model. 
        # We need to convert the datatset from Pandas format (pd.DataFrame)
        # into TensorFlow Datasets format (tf.data.Dataset).
        # TensorFlow Datasets is a high performance data loading library 
        # which is helpful when training neural networks with accelerators like GPUs and TPUs.
        # Note: Some column names contains white spaces at the end of their name, 
        # which is non-comaptible with SavedModels save format. 
        # By default, `pd_dataframe_to_tf_dataset` function will convert 
        # this column names into a compatible format. 
        # So you can safely ignore the warnings related to this.
        train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_df, label=label)
        valid_ds = tfdf.keras.pd_dataframe_to_tf_dataset(valid_df, label=label)

        # Define the model and metrics
        rf = tfdf.keras.RandomForestModel()
        rf.compile(metrics=["accuracy", "binary_crossentropy"]) 
        
        # Train the model
        # We will train the model using a one-liner.
        # Note: you may see a warning about Autograph. 
        # You can safely ignore this, it will be fixed in the next release.
        # Previously calculated class weights is used to handle imbalance.
        rf.fit(x=train_ds, class_weight=class_weight)
        
        # Store the model
        models[f"fold_{i+1}"] = rf
        
        
        # Predict OOF value for validation data
        predict = rf.predict(x=valid_ds)
        
        # Store the predictions in oof dataframe
        oof.loc[valid_ids, 0] = predict.flatten() 
        
        # Evaluate and store the metrics in respective dicts
        evaluation = rf.evaluate(x=valid_ds,return_dict=True)
        accuracy[f"fold_{i+1}"] = evaluation["accuracy"]
        cross_entropy[f"fold_{i+1}"]= evaluation["binary_crossentropy"]

##### Fold 1
Use /tmp/tmpk1jhvvpd as temporary training directory
Reading training dataset...
Training dataset read in 0:00:01.302607. Found 493 examples.
Training model...
Model trained in 0:00:00.344430
Compiling model...


[INFO 23-06-28 20:00:07.1352 UTC kernel.cc:1242] Loading model from path /tmp/tmpk1jhvvpd/model/ with prefix fab7ba28e8b14ec4
[INFO 23-06-28 20:00:07.1968 UTC decision_forest.cc:660] Model loaded with 300 root(s), 14302 node(s), and 56 input feature(s).
[INFO 23-06-28 20:00:07.1969 UTC abstract_model.cc:1311] Engine "RandomForestOptPred" built
[INFO 23-06-28 20:00:07.1969 UTC kernel.cc:1074] Use fast generic engine


Model compiled.
1/1 [==============================] - 0s 424ms/step - loss: 0.0000e+00 - accuracy: 0.9355 - binary_crossentropy: 0.2126
##### Fold 2
Use /tmp/tmpiqkysw9s as temporary training directory
Reading training dataset...
Training dataset read in 0:00:01.217268. Found 493 examples.
Training model...
Model trained in 0:00:00.336665
Compiling model...


[INFO 23-06-28 20:00:10.3726 UTC kernel.cc:1242] Loading model from path /tmp/tmpiqkysw9s/model/ with prefix c06b6a69c3a24226
[INFO 23-06-28 20:00:10.4318 UTC decision_forest.cc:660] Model loaded with 300 root(s), 13938 node(s), and 56 input feature(s).
[INFO 23-06-28 20:00:10.4319 UTC kernel.cc:1074] Use fast generic engine


Model compiled.
1/1 [==============================] - 0s 411ms/step - loss: 0.0000e+00 - accuracy: 0.9274 - binary_crossentropy: 0.2391
##### Fold 3
Use /tmp/tmpc1dus188 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:01.222762. Found 494 examples.
Training model...
Model trained in 0:00:00.364957
Compiling model...


...

Model compiled.
1/1 [==============================] - 0s 401ms/step - loss: 0.0000e+00 - accuracy: 0.9106 - binary_crossentropy: 0.2734
##### Fold 4
Use /tmp/tmp6_p5dcg_ as temporary training directory
Reading training dataset...
Training dataset read in 0:00:01.143838. Found 494 examples.
Training model...
Model trained in 0:00:00.308934
Compiling model...


[INFO 23-06-28 20:00:16.4533 UTC kernel.cc:1242] Loading model from path /tmp/tmp6_p5dcg_/model/ with prefix 8c13ad7ec2e54b9f
[INFO 23-06-28 20:00:16.5073 UTC decision_forest.cc:660] Model loaded with 300 root(s), 13026 node(s), and 55 input feature(s).
[INFO 23-06-28 20:00:16.5074 UTC kernel.cc:1074] Use fast generic engine


Model compiled.
1/1 [==============================] - 0s 411ms/step - loss: 0.0000e+00 - accuracy: 0.9350 - binary_crossentropy: 0.2667
##### Fold 5
Use /tmp/tmp0nnckmon as temporary training directory
Reading training dataset...
Training dataset read in 0:00:01.120012. Found 494 examples.
Training model...
Model trained in 0:00:00.324307
Compiling model...


[INFO 23-06-28 20:00:19.2758 UTC kernel.cc:1242] Loading model from path /tmp/tmp0nnckmon/model/ with prefix 0dd88ede483f4dbb
[INFO 23-06-28 20:00:19.3352 UTC decision_forest.cc:660] Model loaded with 300 root(s), 14286 node(s), and 56 input feature(s).
[INFO 23-06-28 20:00:19.3352 UTC abstract_model.cc:1311] Engine "RandomForestOptPred" built
[INFO 23-06-28 20:00:19.3353 UTC kernel.cc:1074] Use fast generic engine


Model compiled.
1/1 [==============================] - 0s 399ms/step - loss: 0.0000e+00 - accuracy: 0.9268 - binary_crossentropy: 0.2421


### check the model

In [22]:
for _model in models:
    inspector = models[_model].make_inspector()
    print(_model, inspector.evaluation())

fold_1 Evaluation(num_examples=493, accuracy=0.8609655248075181, loss=0.40282299687553547, rmse=None, ndcg=None, aucs=None, auuc=None, qini=None)
fold_2 Evaluation(num_examples=493, accuracy=0.8775482652355239, loss=0.3799142919045157, rmse=None, ndcg=None, aucs=None, auuc=None, qini=None)
fold_3 Evaluation(num_examples=494, accuracy=0.8625497760186837, loss=0.3609043250753038, rmse=None, ndcg=None, aucs=None, auuc=None, qini=None)
fold_4 Evaluation(num_examples=494, accuracy=0.8681451739754287, loss=0.3856540231248326, rmse=None, ndcg=None, aucs=None, auuc=None, qini=None)
fold_5 Evaluation(num_examples=494, accuracy=0.8551073283081274, loss=0.3760922914574804, rmse=None, ndcg=None, aucs=None, auuc=None, qini=None)


# Submission

In [23]:
test_ds_pd = test_df
test_df_columns = test_ds_pd.columns.tolist()
TEST_FEATURE_COLUMNS = [i for i in FEATURE_COLUMNS \
                        if i in test_df_columns and i != "Class"]
test_ds_pd = test_ds_pd[TEST_FEATURE_COLUMNS]
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd)
predictions = models['fold_1'].predict(test_ds)
n_predictions= [[round(abs(i-1), 8), i] for i in predictions.ravel()]
print(n_predictions)

1/1 [==============================] - 0s 148ms/step
[[0.6666669, 0.3333331], [0.6666669, 0.3333331], [0.6666669, 0.3333331], [0.6666669, 0.3333331], [0.6666669, 0.3333331]]


In [25]:
sample_submission[['class_0', 'class_1']] = n_predictions
sample_submission.to_csv('submission.csv', index=False)